# 05 - NIfTI to DICOM Secondary Capture Conversion

This notebook demonstrates how to convert NIfTI files into a series of DICOM Secondary Capture (SC) objects using the `diffusemri` library. This can be useful for importing processed NIfTI images (like parameter maps, segmentations, or custom visualizations) into Picture Archiving and Communication Systems (PACS) or DICOM viewers that might not directly support NIfTI.

**<font color='red'>IMPORTANT WARNING:</font>**
*   **Non-Diagnostic Use:** DICOM Secondary Capture objects are **NOT** intended for primary diagnostic interpretation. They do not contain the rich metadata of original scanner-acquired DICOMs (e.g., detailed acquisition parameters, sequence information). 
*   **Loss of Original Metadata:** The conversion process creates new DICOM objects with minimal necessary metadata. Most of the original NIfTI header information beyond basic spatial details might not be directly transferable or represented in the same way in DICOM SC.
*   **Pixel Data Representation:** Careful attention must be paid to how pixel data from the NIfTI is scaled and represented in the DICOM SC objects, especially if the NIfTI contains floating-point data or values outside the typical range of DICOM display.

In [ ]:
import os
import shutil
import numpy as np
import nibabel as nib
import pydicom # For verifying DICOM output

# diffusemri library imports
from data_io.dicom_utils import write_nifti_to_dicom_secondary

# Setup a temporary directory for example files
TEMP_DIR = "temp_nii2dicom_sec_example"
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
os.makedirs(TEMP_DIR)

print(f"Temporary directory for examples: {os.path.abspath(TEMP_DIR)}")

## Part 1: Creating a Dummy NIfTI File

First, we'll create a simple 3D NIfTI file to use as input for the conversion.

In [ ]:
# Create a dummy 3D NIfTI image with integer data
# Data ranges from 0 up to (6*7*4 - 1) = 167
nifti_data_3d = np.arange(6*7*4, dtype=np.int16).reshape((6, 7, 4)) # Small X, Y, Z dimensions

# Create a simple affine matrix (RAS orientation, 2mm isotropic voxels, origin at [10,20,30])
affine_3d = np.array([
    [2.0, 0.0, 0.0, 10.0],
    [0.0, 2.0, 0.0, 20.0],
    [0.0, 0.0, 2.0, 30.0],
    [0.0, 0.0, 0.0, 1.0]
])

dummy_nifti_3d_filepath = os.path.join(TEMP_DIR, "dummy_3d_image.nii.gz")
nifti_3d_img = nib.Nifti1Image(nifti_data_3d, affine_3d)
nib.save(nifti_3d_img, dummy_nifti_3d_filepath)

print(f"Created dummy 3D NIfTI: {dummy_nifti_3d_filepath}")
print(f"  Data shape: {nifti_3d_img.shape}, Data type: {nifti_3d_img.get_data_dtype()}")
print(f"  Data min: {nifti_data_3d.min()}, Data max: {nifti_data_3d.max()}")

## Part 2: Converting NIfTI to DICOM Secondary Capture Series

Now, we'll use the `write_nifti_to_dicom_secondary` function to convert this NIfTI file into a series of DICOM SC files. Each slice of the NIfTI image will become a separate DICOM file.

In [ ]:
output_dicom_series_dir_3d = os.path.join(TEMP_DIR, "my_dicom_sc_series_3d")
series_description_3d = "Converted 3D NIfTI Data"

print(f"Converting NIfTI file: {dummy_nifti_3d_filepath}")
print(f"Outputting DICOM series to: {output_dicom_series_dir_3d}")

created_dicom_files_list = []
try:
    # The function write_nifti_to_dicom_secondary handles loading the NIfTI file from the path.
    # It will also handle basic data scaling to fit into DICOM pixel representation (e.g., uint16).
    created_dicom_files_list = write_nifti_to_dicom_secondary(
        nifti_image_path=dummy_nifti_3d_filepath, 
        output_dir=output_dicom_series_dir_3d,
        series_description=series_description_3d,
        patient_id="P001-Nii2DCM",
        series_number=77
        # Note: The underlying function will attempt to scale float data to uint16, 
        # or use original integer type if it's uint8/int16/uint16. 
        # Explicit RescaleSlope/Intercept tags are not set by this function for custom scaling by default.
    )
    
    if created_dicom_files_list and len(created_dicom_files_list) > 0:
        print(f"NIfTI file converted successfully to DICOM SC series.")
        print(f"Generated {len(created_dicom_files_list)} DICOM files in {output_dicom_series_dir_3d}")
        print(f"First few files: {created_dicom_files_list[:5]}")
    else:
        print("NIfTI to DICOM conversion reported success, but no files were listed as created or the list was empty.")

except Exception as e:
    print(f"An error occurred during NIfTI to DICOM conversion: {e}")

### Inspecting a Generated DICOM File

Let's load one of the generated DICOM files and inspect some of its key tags to understand what information has been written.

In [ ]:
if created_dicom_files_list and len(created_dicom_files_list) > 0:
    first_dicom_slice_path = created_dicom_files_list[0]
    if os.path.exists(first_dicom_slice_path):
        try:
            ds = pydicom.dcmread(first_dicom_slice_path)
            
            print(f"\n--- Tags from DICOM file: {os.path.basename(first_dicom_slice_path)} ---")
            print(f"SOPClassUID: {ds.SOPClassUID} ({ds.SOPClassUID.name})")
            print(f"Modality: {ds.Modality}") # Should be 'OT' (Other) for Secondary Capture
            print(f"SeriesDescription: {ds.SeriesDescription}")
            print(f"InstanceNumber: {ds.InstanceNumber}")
            print(f"PatientName: {ds.PatientName}")
            print(f"PatientID: {ds.PatientID}")
            print(f"StudyInstanceUID: {ds.StudyInstanceUID}")
            print(f"SeriesInstanceUID: {ds.SeriesInstanceUID}")
            print(f"SeriesNumber: {ds.SeriesNumber}")
            
            print(f"\n--- Spatial and Pixel Information ---")
            print(f"Rows: {ds.Rows}, Columns: {ds.Columns}")
            print(f"PixelSpacing: {ds.PixelSpacing}") # Derived from NIfTI affine
            print(f"SliceThickness: {ds.SliceThickness}") # Derived from NIfTI affine
            print(f"ImagePositionPatient: {ds.ImagePositionPatient}") # Derived from NIfTI affine
            print(f"ImageOrientationPatient: {ds.ImageOrientationPatient}") # Derived from NIfTI affine
            
            print(f"\n--- Pixel Data Representation --- ")
            print(f"BitsAllocated: {ds.BitsAllocated}")
            print(f"BitsStored: {ds.BitsStored}")
            print(f"HighBit: {ds.HighBit}")
            print(f"PixelRepresentation: {ds.PixelRepresentation} (0=unsigned, 1=signed)")
            print(f"PhotometricInterpretation: {ds.PhotometricInterpretation}")
            # Check for Rescale Slope/Intercept if the function sets them (it might not by default for SC)
            if 'RescaleIntercept' in ds:
                print(f"RescaleIntercept: {ds.RescaleIntercept}")
            if 'RescaleSlope' in ds:
                print(f"RescaleSlope: {ds.RescaleSlope}")
            
            # You can also access the pixel data itself via ds.pixel_array
            # print(f"Pixel array shape: {ds.pixel_array.shape}, dtype: {ds.pixel_array.dtype}")
            # print(f"Pixel data min: {ds.pixel_array.min()}, max: {ds.pixel_array.max()}")

        except Exception as e:
            print(f"Error reading or printing tags from generated DICOM file: {e}")
    else:
        print(f"The listed DICOM file does not exist: {first_dicom_slice_path}")
else:
    print("No DICOM files were listed as created, skipping inspection.")

## Important Notes on Usage

*   **Non-Diagnostic Nature**: As highlighted earlier, these DICOM SC files are for informational or secondary review purposes only, not for primary diagnosis. They lack the detailed provenance of original scanner data.

*   **Data Scaling and Pixel Representation**: 
    *   The `write_nifti_to_dicom_secondary` function attempts to handle pixel data scaling appropriately. 
    *   If your input NIfTI data is floating-point, it will be scaled to fit within the `uint16` range (0-65535) for DICOM storage. This involves a min-max normalization. 
    *   If your input NIfTI is an integer type (like `int16`, `uint8`, `uint16`), the function will try to preserve these values. However, if values are outside the `uint16` range (e.g. negative values in `int16` if not shifted, or values > 65535), they might be clipped or an error could occur depending on the exact internal logic for that data type.
    *   The DICOM files will typically have `BitsAllocated = 16`, `BitsStored = 16` (or 8 if input was `uint8`), and `PixelRepresentation = 0` (unsigned). 
    *   The function **does not** currently allow you to set custom `RescaleSlope` and `RescaleIntercept` values in the output DICOM tags for precise control over how a DICOM viewer might interpret stored pixel values. The goal is to store a visual representation. If specific intensity mappings are critical, you might need to pre-scale your NIfTI data appropriately before conversion or use more specialized DICOM creation tools.

*   **Orientation and Spatial Information**: The spatial information in the DICOM SC files (like `ImagePositionPatient`, `ImageOrientationPatient`, `PixelSpacing`, `SliceThickness`) is derived directly from the affine transformation matrix of the input NIfTI file. Ensure your NIfTI file has correct and well-defined orientation information if accurate spatial representation in DICOM is required.

*   **Multi-Frame vs. Series**: This function creates a *series* of single-frame DICOM instances (one file per slice per time point for 4D NIfTI). It does not create multi-frame DICOM objects.

## Cleanup

Remove the temporary directory and its contents.

In [ ]:
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
    print(f"Cleaned up temporary directory: {TEMP_DIR}")